## Precarga de librerias y funciones

In [1]:
%run "../../recurrentes.ipynb"

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.



In [2]:
%run "../../recurrentes.ipynb"

Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Note: you may need to restart the kernel to use updated packages.


#### Excel

In [4]:
# 2 - Cargar datos
# Opciones:
# a) dataset_clase_ternaria_l
# b) dataset_lags_clase_ternaria_l
# c) dataset_lags_deltas_y_clase_ternaria_l
# d) dataset_10_meses_l
# 
df_train = pd.read_parquet(dataset_lags_deltas_y_clase_ternaria_l)



In [5]:
df_train.drop(columns=dicc_psi['dataset_lags_deltas_y_clase_ternaria'], inplace = True)
df_train.drop(columns=dicc_psi['lista_light_gbm_feature_importance'],inplace = True)

In [6]:
df_train.shape

(4735593, 296)

## Formateo pre modelo

In [7]:
if 'T_Visa_normal' in df_train.columns:
    df_train['T_Visa_normal'] = df_train['T_Visa_normal'].astype(bool)
if 'T_Master_normal'in df_train.columns:
    df_train['T_Master_normal'] = df_train['T_Master_normal'].astype(bool)


In [8]:
# 3 - parametros para modelo
ganancia_acierto = 273000
costo_estimulo = 7000

mes_train = 202106
mes_test = 202108


In [9]:
data = df_train

In [10]:

data['clase_peso'] = 1.0
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001
data['clase_binaria'] = np.where(data['clase_ternaria']=='CONTINUA', 0, 1)
data['foto_mes'].unique()
meses = [202011, 202012, 202101, 202102, 202103, 202104,202105,202106]


df_train = data[data['foto_mes']<=mes_train]
df_test = data[data['foto_mes']==mes_test]
clase_peso = df_train['clase_peso']
X_train = df_train.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_train =df_train['clase_binaria']
X_test = df_test.drop(['clase_ternaria', 'clase_binaria', 'clase_peso'], axis=1)
Y_test =df_test['clase_binaria']
w_train = df_train['clase_peso']
w_test = df_test['clase_peso']
df_train['foto_mes'].unique()

array([201901, 201902, 201903, 201904, 201905, 201906, 201907, 201908,
       201909, 201910, 201911, 201912, 202001, 202002, 202003, 202004,
       202005, 202006, 202007, 202008, 202009, 202010, 202011, 202012,
       202101, 202102, 202103, 202104, 202105, 202106], dtype=int64)

## Predicciones

In [11]:
best_params = {'num_leaves': 1796,
 'learning_rate': 0.049283676079631966,
 'min_data_in_leaf': 9,
 'feature_fraction': 0.44425115760554584,
 'bagging_fraction': 0.40692971865213867}

In [ ]:
# best_params = {'num_leaves': 3545,
#  'learning_rate': 0.07959540527561224,
#  'min_data_in_leaf': 933,
#  'feature_fraction': 0.6829932939284065,
#  'bagging_fraction': 0.6251908881121132}
# best_iter = 5

In [85]:
numeros_random = np.random.randint(0, 100000, 20)

In [12]:


train_data = lgb.Dataset(X_train,
                            label=Y_train,
                            weight=w_train)



best_iter = 1789
df_modelos = pd.DataFrame
params = {
'objective': 'binary',
'boosting_type': 'gbdt',
'first_metric_only': True,
'boost_from_average': True,
'feature_pre_filter': False,
'max_bin': 31,
'num_leaves': best_params['num_leaves'],
'learning_rate': best_params['learning_rate'],
'min_data_in_leaf': best_params['min_data_in_leaf'],
'feature_fraction': best_params['feature_fraction'],
'bagging_fraction': best_params['bagging_fraction'],
'seed': semillas[0],
'verbose': 0
}
        
        
model = lgb.train(params,
                train_data,
                num_boost_round=best_iter)

y_pred = model.predict(X_test)



MemoryError: Unable to allocate 9.68 GiB for an array with shape (4404999, 295) and data type float64

In [ ]:

X_test['Probabilidad'] = y_pred

tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)

tb_entrega['Predicted'] = 0

envios = 12300
tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1

resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)

print("Cantidad de clientes {}".format(envios))
num_subida_kaggle = 27
nombre_archivo = '\entrega_0{}.csv'.format(num_subida_kaggle)
entrega_final = f'{save_path}{nombre_archivo}'
resultados.to_csv(entrega_final, index=False)

Cantidad de clientes 14100


In [ ]:
# bajas = pd.DataFrame({'numero_de_cliente': X_test['numero_de_cliente'], 'clase_ternaria': df_test['clase_ternaria']}, index= X_test.index)

In [ ]:
# prediccion.isna().sum()

numero_de_cliente    0
Probabilidad         0
dtype: int64

## Punto de corte

In [ ]:
# merged_data = prediccion.merge(bajas, on='numero_de_cliente', how='inner')


In [ ]:
# merged_data_sorted = merged_data.sort_values('Probabilidad', ascending=False)


In [ ]:

# # Ordenar por probabilidad de mayor a menor

# # Seleccionar top clientes
# top_clients = merged_data_sorted.iloc[:15000].copy()  # Trabajar con una copia

# # Convertir 'clase_ternaria' a variable binaria para los top clientes
# top_clients.loc[:, 'bajas_reales'] = (top_clients['clase_ternaria'] == 'BAJA+2').astype(int)

# top_clients.value_counts('clase_ternaria')

clase_ternaria
BAJA+1    15000
Name: count, dtype: int64

In [ ]:
# def calculoGanancia(bajas,prediccion,corte):
#     ''' 
#     Calcula la ganancia para una semilla específica.
    
#     Parámetros:
#     bajas: DataFrame con columnas "numero_de_cliente" y "clase_ternaria".
#     prediccion: DataFrame con columnas "numero_de_cliente" y "Probabilidad".
#     corte: int, cantidad de estímulos.
#     random_state: int, semilla para train_test_split.
    
#     Retorna:
#     ganancia_publico: Ganancia para el público.
#     ganancia_privado: Ganancia para el privado.
#     '''
#     # Realizar el split en público y privado
#     Publico, Privado = train_test_split(
#         bajas,
#         test_size=0.7,
#         stratify=bajas['clase_ternaria'],
#         random_state=123
#     )

#     # Clientes que decido estimular
#     estimulos = prediccion.iloc[:corte] 

#     # Obtener los estímulos en el conjunto público y privado
#     estimulos_publico = pd.merge(estimulos, Publico, on='numero_de_cliente', how='inner')
#     estimulos_privado = pd.merge(estimulos, Privado, on='numero_de_cliente', how='inner')

#     # Calcular los verdaderos positivos en cada conjunto
#     TP_publico = estimulos_publico[estimulos_publico['clase_ternaria'] == 'BAJA+2']
#     TP_privado = estimulos_privado[estimulos_privado['clase_ternaria'] == 'BAJA+2']

#     # 5. Calcular la ganancia para cada conjunto con normalización
#     # Primero, calculamos la ganancia en cada conjunto
#     ganancia_publico_sin_norm = (len(TP_publico) * 273000) - ((len(estimulos_publico) - len(TP_publico)) * 7000)
#     ganancia_privado_sin_norm = (len(TP_privado) * 273000) - ((len(estimulos_privado) - len(TP_privado)) * 7000)

#     # Luego, normalizamos dividiendo por el porcentaje correspondiente
#     ganancia_publico = ganancia_publico_sin_norm / 0.3
#     ganancia_privado = ganancia_privado_sin_norm / 0.7

#     return ganancia_publico, ganancia_privado


In [ ]:
# inicio = time.time()

# resultados = []
# cortes = range(5000, 20000, 1000)


# pred_model_sorted = prediccion.sort_values('Probabilidad', ascending=False)
# model_name = 'LightGBM'

# # Iteramos sobre cada corte
# for corte in cortes:
#     ganancia_publico, ganancia_privado = calculoGanancia(bajas, pred_model_sorted, corte)
    
#     # Almacenamos los resultados
#     resultados.append({
#         'Modelo': model_name,
#         'Corte': corte,
#         'Ganancia Público': ganancia_publico,
#         'Ganancia Privado': ganancia_privado
#     })

# # Convertimos los resultados en un DataFrame
# resultados = pd.DataFrame(resultados)

# fin = time.time()
# tiempo_ejecucion = fin - inicio

# print(f"Tiempo de ejecución: {tiempo_ejecucion:.2f} segundos")

Tiempo de ejecución: 1.75 segundos


In [ ]:
# # Pivotamos el DataFrame 'resultados' para reorganizar las ganancias
# resultados_pivot = resultados.pivot_table(
#     index='Corte',
#     columns='Modelo',
#     values=['Ganancia Público', 'Ganancia Privado']
# )

# # Aplanamos las columnas para facilitar el acceso
# resultados_pivot.columns = [f'{ganancia}_{modelo}' for ganancia, modelo in resultados_pivot.columns]

# # Reordenamos las columnas alternando 'Público' y 'Privado' para cada modelo
# # Ordenamos primero por el modelo, luego alternando entre 'Público' y 'Privado'
# columnas_ordenadas = []
# for modelo in resultados['Modelo'].unique():
#     columnas_ordenadas.append(f'Ganancia Público_{modelo}')
#     columnas_ordenadas.append(f'Ganancia Privado_{modelo}')

# # Reorganizamos el DataFrame usando el nuevo orden de columnas
# resultados_pivot = resultados_pivot[columnas_ordenadas]

# # Convertimos el índice 'Corte' en una columna si prefieres tenerla como tal
# resultados_pivot = resultados_pivot.reset_index()
# resultados_pivot

,Corte,Ganancia Público_LightGBM,Ganancia Privado_LightGBM
0,5000,-3.511667e+07,-34950000.0
1,6000,-4.160333e+07,-42170000.0
2,7000,-4.900000e+07,-49000000.0
3,8000,-5.569667e+07,-56130000.0
4,9000,-6.295333e+07,-63020000.0
5,10000,-6.965000e+07,-70150000.0
6,11000,-7.665000e+07,-77150000.0
7,12000,-8.339333e+07,-84260000.0
8,13000,-9.074333e+07,-91110000.0
9,14000,-9.783667e+07,-98070000.0


In [ ]:
# 